In [7]:
import pandas as pd
import ast
import re

In [8]:
df=pd.read_excel("hyderabad_cars.xlsx")

CAR DETAIL

In [9]:

df['new_car_detail'] = df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
new_car_details_df = pd.json_normalize(df['new_car_detail'])
new_car_details_df.insert(0, 'state', 'hyderabad')

In [11]:
new_car_details_df.drop(columns=["it","owner","priceFixedText","priceActual","priceSaving","trendingText.imgUrl",
                          "trendingText.heading","trendingText.desc"],inplace=True)

def convert_numberic(value):
    new=value.replace("₹","").replace(',', '').strip()
    if "Lakh" in new:
        return float(new.replace('Lakh', '').strip())*1e5
    elif "Crore" in new:
        return float(new.replace("Crore","").strip())*1e7
    else:
        return float(new)
new_car_details_df["price"]=new_car_details_df["price"].apply(convert_numberic)

KeyError: "['it', 'owner', 'priceFixedText', 'priceActual', 'priceSaving', 'trendingText.imgUrl', 'trendingText.heading', 'trendingText.desc'] not found in axis"

In [12]:

# Extracting 'top' key values into separate columns
def extract_keys(dict_list):
    result = {}
    for item in dict_list:
        result[item['key']] = item['value']
    return result

# Apply function and expand the dictionary into columns
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
car_overview = df["new_car_overview"].apply(lambda x: extract_keys(x['top'])).apply(pd.Series)


In [14]:

def extract_year(value):
    match = re.search(r'\b\d{4}\b', str(value))
    if match:
        # Extract the year and return it as an integer
        return int(match.group())
    return pd.NA  # Use pd.NA for missing values

# Apply the function to extract the year
car_overview['Registration Year'] = car_overview['Registration Year'].apply(extract_year)

# Convert the column datatype to 'Int64' (nullable integer type)
car_overview['Registration Year'] = car_overview['Registration Year'].astype('Int64')


def extract_number(text):
    # Ensure the input is a string
    if isinstance(text, str):
        match = re.search(r'\d+', text)
        if match:
            return int(match.group())
    return None


car_overview['Seats'] = car_overview['Seats'].apply(extract_number)


import pandas as pd
import re

def extract_kms(text):
    
    if isinstance(text, str):
        
        numeric_value = re.sub(r'[^\d,]', '', text)
        
        return int(numeric_value.replace(',', ''))
    return None


# Apply the function to the 'Distance' column
car_overview['Kms Driven'] = car_overview['Kms Driven'].apply(extract_kms)
car_overview.drop(columns=["Ownership"],inplace=True)

KeyError: "['Ownership'] not found in axis"

In [8]:
car_overview["Registration Year"]=car_overview["Registration Year"].fillna(2019)
car_overview["Seats"]=car_overview["Seats"].fillna(5.0)
car_overview["Kms Driven"]=car_overview["Kms Driven"].fillna(car_overview["Kms Driven"].mean())
car_overview["Year of Manufacture"]=car_overview["Year of Manufacture"].fillna(2019)

CAR FEATURES

In [9]:
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
result=[]
for sec in df["new_car_feature"]:
   
   
    df1 = pd.DataFrame([item['value'] for item in sec["top"]], columns=['Features'])

# Concatenate all the values into a single string and place it in a single row
    concatenated_features = ','.join(df1['Features'])
    result.append({'Features': concatenated_features})

df_car_fearures = pd.DataFrame(result)

CAR SPECIFICATION

In [10]:
df['new_car_specs'] = df['new_car_specs'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

def extract_specs(specs):
   
    # Extract from 'top'
    top_data = extract_keys(specs.get('top', []))
    
    # Extract from 'list' in 'data'
    list_data = {}
    if 'data' in specs:
        for section in specs['data']:
            list_data.update(extract_keys(section.get('list', [])))
    
    # Combine both dictionaries
    combined_data = {**top_data, **list_data}
    return combined_data

car_spec_df = df["new_car_specs"].apply(extract_specs).apply(pd.Series)

In [11]:
car_spec_df.drop(columns=["Ground Clearance Unladen","Seats","Values per Cylinder","Front Tread","Seating Capacity",
                          "Compression Ratio","BoreX Stroke","Gross Weight","Alloy Wheel Size","Rear Tread","Acceleration",
                          "Top Speed","Wheel Size","Value Configuration"
                          ],inplace=True)

def millimeter(value,mini):
    if pd.notna(value) and mini in value:
        value = value.replace(mini, "").strip()
        value = pd.to_numeric(value, errors='coerce')
        return value
    return pd.to_numeric(value, errors='coerce')

def meter(value,mini1,mini2,mini3):
    if pd.notna(value) and mini1 and mini2 and mini3 in value:
        value = value.replace(mini1, "").replace(mini2, "").replace(mini3, "").strip()
        value = pd.to_numeric(value, errors='coerce')
        return value
    return pd.to_numeric(value, errors='coerce')

car_spec_df["Mileage"] = car_spec_df["Mileage"].str.replace(" kmpl", "", regex=True)
car_spec_df["Mileage"] = pd.to_numeric(car_spec_df["Mileage"], errors='coerce')

car_spec_df["Torque"] = car_spec_df["Torque"].str.replace("Nm", "", regex=True).str.replace("nm", "", regex=True)
car_spec_df["Torque"] = pd.to_numeric(car_spec_df["Torque"], errors='coerce')

car_spec_df["Engine"] = car_spec_df["Engine"].str.replace("CC", "", regex=True)
car_spec_df["Engine"] = pd.to_numeric(car_spec_df["Engine"], errors='coerce')

car_spec_df["Displacement"]=pd.to_numeric(car_spec_df["Displacement"], errors='coerce')

car_spec_df["No of Cylinder"] = pd.to_numeric(car_spec_df["No of Cylinder"], errors='coerce')

car_spec_df["No Door Numbers"]=pd.to_numeric(car_spec_df["No Door Numbers"], errors='coerce')

car_spec_df["Cargo Volumn"]=car_spec_df["Cargo Volumn"].apply(lambda x: millimeter(x, "-litre"))

car_spec_df["Turning Radius"]=car_spec_df["Turning Radius"].apply(lambda x:meter(x, "metres","mtrs.","m"))


In [12]:
car_spec_df["Mileage"]=car_spec_df["Mileage"].fillna(19.7)
car_spec_df["Torque"]=car_spec_df["Torque"].fillna(1370)
car_spec_df["No of Cylinder"]=car_spec_df["No of Cylinder"].fillna(4.0)
car_spec_df["Length"]=car_spec_df["Length"].fillna(	4047.6)
car_spec_df["Width"]=car_spec_df["Width"].fillna(1709.4)
car_spec_df["Height"]=car_spec_df["Height"].fillna(1567.4)
car_spec_df["Wheel Base"]=car_spec_df["Wheel Base"].fillna(2523.9)
car_spec_df["Turning Radius"]=car_spec_df["Turning Radius"].fillna(5.0)
car_spec_df["No Door Numbers"]=car_spec_df["No Door Numbers"].fillna(5.0)
car_spec_df["Cargo Volumn"]=car_spec_df["Cargo Volumn"].fillna(355.1)

In [13]:
df = pd.concat([new_car_details_df, car_overview, df_car_fearures, car_spec_df], axis=1)

In [14]:
df

,state,ft,bt,km,transmission,ownerNo,oem,model,modelYear,centralVariantId,...,Kerb Weight,Gear Box,Drive Type,Steering Type,Turning Radius,Front Brake Type,Rear Brake Type,Tyre Type,No Door Numbers,Cargo Volumn
0,hyderabad,Petrol,Hatchback,"20,000",Automatic,1,Volkswagen,Volkswagen Polo,2022,7746,...,1106,6 Speed,FWD,Electronic,4.90,Disc,Drum,"Tubeless,Radial",5,280.0
1,hyderabad,Petrol,Hatchback,"85,582",Manual,2,Hyundai,Hyundai EON,2014,1502,...,880,5 Speed,FWD,Power,4.60,Disc,Drum,Tubeless,5,355.1
2,hyderabad,Petrol,SUV,"10,000",Manual,1,Hyundai,Hyundai Venue,2023,9153,...,1106,5-Speed,NaN,Power,5.00,Disc,Drum,"Tubeless, Radial",5,355.1
3,hyderabad,Petrol,Hatchback,"50,750",Manual,1,Maruti,Maruti Baleno,2017,3970,...,890Kg,5 Speed,FWD,Power,5.00,Disc,Drum,"Tubeless,Radial",5,355.1
4,hyderabad,Petrol,SUV,"10,000",Automatic,1,Mahindra,Mahindra Thar,2022,8074,...,1825,6 Speed,4X4,Power,5.00,Disc,Drum,"Radial, Tubeless",3,355.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478,hyderabad,Diesel,SUV,"60,000",Manual,1,Mahindra,Mahindra XUV500,2013,3823,...,1940,6 Speed,FWD,Power,5.60,Disc & Caliper Type,Disc & Caliper Type,"Tubeless,Radial",5,355.1
1479,hyderabad,Diesel,Sedan,"50,000",Automatic,1,Jaguar,Jaguar XF,2014,5007,...,1780kg,8 Speed,RWD,Power,5.75,Disc,Disc,"Tubeless,Radial",4,355.1
1480,hyderabad,Diesel,Sedan,"1,20,000",Automatic,3,BMW,BMW 5 Series,2018,250,...,1660,8-Speed Steptronic,RWD,Power,5.60,Ventilated Disc,Ventilated Disc,"Tubeless, Runflat",4,355.1
1481,hyderabad,Diesel,SUV,"50,000",Automatic,2,Tata,Tata Nexon,2019,6500,...,1305Kg,6 Speed,FWD,Power,5.10,Disc,Drum,Tubeless Radial Tyres,5,350.0


In [5]:
import pandas as pd
import ast
import re
df=pd.read_excel("C:\project 3\chennai_cars.xlsx")

# new_car_detail

df['new_car_detail'] = df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
new_car_details_df = pd.json_normalize(df['new_car_detail'])
new_car_details_df.insert(0, 'state', 'chennai')
new_car_details_df.drop(columns=["it","owner","priceFixedText","priceActual","priceSaving","trendingText.imgUrl",
                          "trendingText.heading","trendingText.desc"],inplace=True)

def convert_numberic(value):
    new=value.replace("₹","").replace(',', '').strip()
    if "Lakh" in new:
        return float(new.replace('Lakh', '').strip())*1e5
    elif "Crore" in new:
        return float(new.replace("Crore","").strip())*1e7
    else:
        return float(new)
new_car_details_df["price"]=new_car_details_df["price"].apply(convert_numberic)

# car_overview


# Extracting 'top' key values into separate columns
def extract_keys(dict_list):
    result = {}
    for item in dict_list:
        result[item['key']] = item['value']
    return result

# Apply function and expand the dictionary into columns
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
car_overview = df["new_car_overview"].apply(lambda x: extract_keys(x['top'])).apply(pd.Series)

#convert dataformat


def extract_year(value):
    match = re.search(r'\b\d{4}\b', str(value))
    if match:
        # Extract the year and return it as an integer
        return int(match.group())
    return pd.NA  # Use pd.NA for missing values

# Apply the function to extract the year
car_overview['Registration Year'] = car_overview['Registration Year'].apply(extract_year)

# Convert the column datatype to 'Int64' (nullable integer type)
car_overview['Registration Year'] = car_overview['Registration Year'].astype('Int64')


def extract_number(text):
    # Ensure the input is a string
    if isinstance(text, str):
        match = re.search(r'\d+', text)
        if match:
            return int(match.group())
    return None


car_overview['Seats'] = car_overview['Seats'].apply(extract_number)


import pandas as pd
import re

def extract_kms(text):
    
    if isinstance(text, str):
        
        numeric_value = re.sub(r'[^\d,]', '', text)
        
        return int(numeric_value.replace(',', ''))
    return None


# Apply the function to the 'Distance' column
car_overview['Kms Driven'] = car_overview['Kms Driven'].apply(extract_kms)
car_overview.drop(columns=["Ownership"],inplace=True)
#null imputation integers and float
car_overview["Registration Year"]=car_overview["Registration Year"].fillna(2019)
car_overview["Seats"]=car_overview["Seats"].fillna(5.0)
car_overview["Kms Driven"]=car_overview["Kms Driven"].fillna(car_overview["Kms Driven"].mean())
car_overview["Year of Manufacture"]=car_overview["Year of Manufacture"].fillna(2019)
# df_car_fearures
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
result=[]
for sec in df["new_car_feature"]:
   
   
    df1 = pd.DataFrame([item['value'] for item in sec["top"]], columns=['Features'])

# Concatenate all the values into a single string and place it in a single row
    concatenated_features = ','.join(df1['Features'])
    result.append({'Features': concatenated_features})

df_car_fearures = pd.DataFrame(result)
# new_car_specs
df['new_car_specs'] = df['new_car_specs'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

def extract_specs(specs):
   
    # Extract from 'top'
    top_data = extract_keys(specs.get('top', []))
    
    # Extract from 'list' in 'data'
    list_data = {}
    if 'data' in specs:
        for section in specs['data']:
            list_data.update(extract_keys(section.get('list', [])))
    
    # Combine both dictionaries
    combined_data = {**top_data, **list_data}
    return combined_data

car_spec_df = df["new_car_specs"].apply(extract_specs).apply(pd.Series)
#convert dataformat
car_spec_df.drop(columns=["Ground Clearance Unladen","Seats","Values per Cylinder","Front Tread","Seating Capacity",
                          "Compression Ratio","BoreX Stroke","Gross Weight","Alloy Wheel Size","Rear Tread","Acceleration",
                          "Top Speed","Wheel Size","Value Configuration"
                          ],inplace=True)

def millimeter(value,mini):
    if pd.notna(value) and mini in value:
        value = value.replace(mini, "").strip()
        value = pd.to_numeric(value, errors='coerce')
        return value
    return pd.to_numeric(value, errors='coerce')

def meter(value,mini1,mini2,mini3):
    if pd.notna(value) and mini1 and mini2 and mini3 in value:
        value = value.replace(mini1, "").replace(mini2, "").replace(mini3, "").strip()
        value = pd.to_numeric(value, errors='coerce')
        return value
    return pd.to_numeric(value, errors='coerce')

car_spec_df["Mileage"] = car_spec_df["Mileage"].str.replace(" kmpl", "", regex=True)
car_spec_df["Mileage"] = pd.to_numeric(car_spec_df["Mileage"], errors='coerce')

car_spec_df["Torque"] = car_spec_df["Torque"].str.replace("Nm", "", regex=True).str.replace("nm", "", regex=True)
car_spec_df["Torque"] = pd.to_numeric(car_spec_df["Torque"], errors='coerce')

car_spec_df["Engine"] = car_spec_df["Engine"].str.replace("CC", "", regex=True)
car_spec_df["Engine"] = pd.to_numeric(car_spec_df["Engine"], errors='coerce')

car_spec_df["Displacement"]=pd.to_numeric(car_spec_df["Displacement"], errors='coerce')

car_spec_df["No of Cylinder"] = pd.to_numeric(car_spec_df["No of Cylinder"], errors='coerce')

car_spec_df["No Door Numbers"]=pd.to_numeric(car_spec_df["No Door Numbers"], errors='coerce')

car_spec_df["Cargo Volumn"]=car_spec_df["Cargo Volumn"].apply(lambda x: millimeter(x, "-litre"))

car_spec_df["Turning Radius"]=car_spec_df["Turning Radius"].apply(lambda x:meter(x, "metres","mtrs.","m"))


#null imputation
car_spec_df["Mileage"]=car_spec_df["Mileage"].fillna(19.7)
car_spec_df["Torque"]=car_spec_df["Torque"].fillna(1370)
car_spec_df["No of Cylinder"]=car_spec_df["No of Cylinder"].fillna(4.0)
car_spec_df["Length"]=car_spec_df["Length"].fillna(	4047.6)
car_spec_df["Width"]=car_spec_df["Width"].fillna(1709.4)
car_spec_df["Height"]=car_spec_df["Height"].fillna(1567.4)
car_spec_df["Wheel Base"]=car_spec_df["Wheel Base"].fillna(2523.9)
car_spec_df["Turning Radius"]=car_spec_df["Turning Radius"].fillna(5.0)
car_spec_df["No Door Numbers"]=car_spec_df["No Door Numbers"].fillna(5.0)
car_spec_df["Cargo Volumn"]=car_spec_df["Cargo Volumn"].fillna(355.1)
dfc = pd.concat([new_car_details_df, car_overview, df_car_fearures, car_spec_df], axis=1)

dfc.to_csv("chennainew.csv")
dfc=pd.read_csv("chennainew.csv")
dfc

<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
C:\Users\reach\AppData\Local\Temp\ipykernel_6936\248902408.py:4: SyntaxWarning: invalid escape sequence '\p'
  df=pd.read_excel("C:\project 3\chennai_cars.xlsx")


,Unnamed: 0,state,ft,bt,km,transmission,ownerNo,oem,model,modelYear,...,Drive Type,Steering Type,Front Brake Type,Rear Brake Type,Tyre Type,No Door Numbers,Cargo Volumn,Mileage,Super Charger,Turning Radius
0,0,chennai,Petrol,SUV,"20,000",Automatic,1,Kia,Kia Sonet,2022,...,FWD,Electric,Disc,Drum,"Tubeless, Radial",5,392.0,19.70,NaN,5.0
1,1,chennai,Petrol,Minivans,"20,687",Manual,1,Maruti,Maruti Eeco,2015,...,RWD,Manual,Ventilated Disc,Drum,Tubeless Tyres,5,355.1,15.37,No,4.5
2,2,chennai,Petrol,SUV,"30,000",Manual,1,Nissan,Nissan Magnite,2021,...,NaN,Electronic,Disc,Drum,"Tubeless,Radial",5,336.0,20.00,No,5.0
3,3,chennai,Petrol,Hatchback,"59,247",Manual,1,Hyundai,Hyundai i10,2015,...,FWD,Power,Ventilated Disc,Drum,Tubeless,5,355.1,19.81,No,4.7
4,4,chennai,Petrol,Hatchback,"50,000",Manual,1,Honda,Honda Jazz,2015,...,FWD,Power,Disc,Drum,"Tubeless,Radial",5,355.1,18.70,No,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,1414,chennai,Petrol,Hatchback,"42,891",Automatic,1,Maruti,Maruti Swift,2018,...,NaN,Power,Disc,Drum,Tubeless,5,355.1,22.00,No,5.0
1415,1415,chennai,Diesel,SUV,"59,100",Manual,1,Renault,Renault Duster,2018,...,FWD,Power,Disc,Drum,"Tubeless,Radial",5,355.1,19.87,No,5.2
1416,1416,chennai,Petrol,SUV,"55,913",Automatic,1,Tata,Tata Nexon,2018,...,FWD,Power,Disc,Drum,Tubeless Radial Tyres,5,350.0,17.00,No,5.1
1417,1417,chennai,Diesel,SUV,"65,000",Automatic,1,Volkswagen,Volkswagen Tiguan,2017,...,AWD,Electric,Disc,Disc,Tubeless Radial Tyres,5,615.0,16.65,No,5.0


In [6]:
import pandas as pd
import ast
import re
df=pd.read_excel("C:\project 3\jaipur_cars.xlsx")

# new_car_detail

df['new_car_detail'] = df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
new_car_details_df = pd.json_normalize(df['new_car_detail'])
new_car_details_df.insert(0, 'state', 'jaipur')
new_car_details_df.drop(columns=["it","owner","priceFixedText","priceActual","priceSaving","trendingText.imgUrl",
                          "trendingText.heading","trendingText.desc"],inplace=True)

def convert_numberic(value):
    new=value.replace("₹","").replace(',', '').strip()
    if "Lakh" in new:
        return float(new.replace('Lakh', '').strip())*1e5
    elif "Crore" in new:
        return float(new.replace("Crore","").strip())*1e7
    else:
        return float(new)
new_car_details_df["price"]=new_car_details_df["price"].apply(convert_numberic)

# car_overview


# Extracting 'top' key values into separate columns
def extract_keys(dict_list):
    result = {}
    for item in dict_list:
        result[item['key']] = item['value']
    return result

# Apply function and expand the dictionary into columns
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
car_overview = df["new_car_overview"].apply(lambda x: extract_keys(x['top'])).apply(pd.Series)

#convert dataformat


def extract_year(value):
    match = re.search(r'\b\d{4}\b', str(value))
    if match:
        # Extract the year and return it as an integer
        return int(match.group())
    return pd.NA  # Use pd.NA for missing values

# Apply the function to extract the year
car_overview['Registration Year'] = car_overview['Registration Year'].apply(extract_year)

# Convert the column datatype to 'Int64' (nullable integer type)
car_overview['Registration Year'] = car_overview['Registration Year'].astype('Int64')


def extract_number(text):
    # Ensure the input is a string
    if isinstance(text, str):
        match = re.search(r'\d+', text)
        if match:
            return int(match.group())
    return None


car_overview['Seats'] = car_overview['Seats'].apply(extract_number)


import pandas as pd
import re

def extract_kms(text):
    
    if isinstance(text, str):
        
        numeric_value = re.sub(r'[^\d,]', '', text)
        
        return int(numeric_value.replace(',', ''))
    return None


# Apply the function to the 'Distance' column
car_overview['Kms Driven'] = car_overview['Kms Driven'].apply(extract_kms)
car_overview.drop(columns=["Ownership"],inplace=True)
#null imputation integers and float
car_overview["Registration Year"]=car_overview["Registration Year"].fillna(2019)
car_overview["Seats"]=car_overview["Seats"].fillna(5.0)
car_overview["Kms Driven"]=car_overview["Kms Driven"].fillna(car_overview["Kms Driven"].mean())
car_overview["Year of Manufacture"]=car_overview["Year of Manufacture"].fillna(2019)
# df_car_fearures
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
result=[]
for sec in df["new_car_feature"]:
   
   
    df1 = pd.DataFrame([item['value'] for item in sec["top"]], columns=['Features'])

# Concatenate all the values into a single string and place it in a single row
    concatenated_features = ','.join(df1['Features'])
    result.append({'Features': concatenated_features})

df_car_fearures = pd.DataFrame(result)
# new_car_specs
df['new_car_specs'] = df['new_car_specs'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

def extract_specs(specs):
   
    # Extract from 'top'
    top_data = extract_keys(specs.get('top', []))
    
    # Extract from 'list' in 'data'
    list_data = {}
    if 'data' in specs:
        for section in specs['data']:
            list_data.update(extract_keys(section.get('list', [])))
    
    # Combine both dictionaries
    combined_data = {**top_data, **list_data}
    return combined_data

car_spec_df = df["new_car_specs"].apply(extract_specs).apply(pd.Series)
#convert dataformat
car_spec_df.drop(columns=["Ground Clearance Unladen","Seats","Values per Cylinder","Front Tread","Seating Capacity",
                          "Compression Ratio","BoreX Stroke","Gross Weight","Alloy Wheel Size","Rear Tread","Acceleration",
                          "Top Speed","Wheel Size","Value Configuration"
                          ],inplace=True)

def millimeter(value,mini):
    if pd.notna(value) and mini in value:
        value = value.replace(mini, "").strip()
        value = pd.to_numeric(value, errors='coerce')
        return value
    return pd.to_numeric(value, errors='coerce')

def meter(value,mini1,mini2,mini3):
    if pd.notna(value) and mini1 and mini2 and mini3 in value:
        value = value.replace(mini1, "").replace(mini2, "").replace(mini3, "").strip()
        value = pd.to_numeric(value, errors='coerce')
        return value
    return pd.to_numeric(value, errors='coerce')

car_spec_df["Mileage"] = car_spec_df["Mileage"].str.replace(" kmpl", "", regex=True)
car_spec_df["Mileage"] = pd.to_numeric(car_spec_df["Mileage"], errors='coerce')

car_spec_df["Torque"] = car_spec_df["Torque"].str.replace("Nm", "", regex=True).str.replace("nm", "", regex=True)
car_spec_df["Torque"] = pd.to_numeric(car_spec_df["Torque"], errors='coerce')

car_spec_df["Engine"] = car_spec_df["Engine"].str.replace("CC", "", regex=True)
car_spec_df["Engine"] = pd.to_numeric(car_spec_df["Engine"], errors='coerce')

car_spec_df["Displacement"]=pd.to_numeric(car_spec_df["Displacement"], errors='coerce')

car_spec_df["No of Cylinder"] = pd.to_numeric(car_spec_df["No of Cylinder"], errors='coerce')

car_spec_df["No Door Numbers"]=pd.to_numeric(car_spec_df["No Door Numbers"], errors='coerce')

car_spec_df["Cargo Volumn"]=car_spec_df["Cargo Volumn"].apply(lambda x: millimeter(x, "-litre"))

car_spec_df["Turning Radius"]=car_spec_df["Turning Radius"].apply(lambda x:meter(x, "metres","mtrs.","m"))


#null imputation
car_spec_df["Mileage"]=car_spec_df["Mileage"].fillna(19.7)
car_spec_df["Torque"]=car_spec_df["Torque"].fillna(1370)
car_spec_df["No of Cylinder"]=car_spec_df["No of Cylinder"].fillna(4.0)
car_spec_df["Length"]=car_spec_df["Length"].fillna(	4047.6)
car_spec_df["Width"]=car_spec_df["Width"].fillna(1709.4)
car_spec_df["Height"]=car_spec_df["Height"].fillna(1567.4)
car_spec_df["Wheel Base"]=car_spec_df["Wheel Base"].fillna(2523.9)
car_spec_df["Turning Radius"]=car_spec_df["Turning Radius"].fillna(5.0)
car_spec_df["No Door Numbers"]=car_spec_df["No Door Numbers"].fillna(5.0)
car_spec_df["Cargo Volumn"]=car_spec_df["Cargo Volumn"].fillna(355.1)
dfj = pd.concat([new_car_details_df, car_overview, df_car_fearures, car_spec_df], axis=1)

dfj.to_csv("jaipurnew.csv")
dfj=pd.read_csv("jaipurnew.csv")
dfj

<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
C:\Users\reach\AppData\Local\Temp\ipykernel_6936\638199267.py:4: SyntaxWarning: invalid escape sequence '\p'
  df=pd.read_excel("C:\project 3\jaipur_cars.xlsx")


,Unnamed: 0,state,ft,bt,km,transmission,ownerNo,oem,model,modelYear,...,Kerb Weight,Gear Box,Drive Type,Steering Type,Turning Radius,Front Brake Type,Rear Brake Type,Tyre Type,No Door Numbers,Cargo Volumn
0,0,jaipur,Diesel,Hatchback,"1,20,000",Manual,2,Hyundai,Hyundai i20,2015,...,1515kg,6 Speed,FWD,Power,5.20,Disc,Drum,Tubeless,5.0,355.1
1,1,jaipur,Petrol,Hatchback,"66,951",Manual,1,Maruti,Maruti Swift,2012,...,875-905,5-Speed,NaN,Electric,4.80,Disc,Drum,"Radial, Tubeless",5.0,268.0
2,2,jaipur,Petrol,Hatchback,"80,000",Automatic,2,Maruti,Maruti Celerio,2016,...,840kg,5 Speed,FWD,Power,4.70,Ventilated Disc,Drum,"Tubeless, Radial",5.0,355.1
3,3,jaipur,Petrol,Hatchback,"44,392",Manual,1,Hyundai,Hyundai Grand i10,2017,...,1100,5 Speed,FWD,Power,4.80,Disc,Drum,Tubeless,5.0,256.0
4,4,jaipur,Petrol,Hatchback,"40,000",Automatic,1,Maruti,Maruti Wagon R,2016,...,890kg,5 Speed,FWD,Power,4.60,Ventilated Disc,Drum,Tubeless Tyres,5.0,355.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,1115,jaipur,Petrol,Hatchback,"1,20,000",Manual,1,Chevrolet,Chevrolet Beat,2011,...,990,5 Speed,FWD,Power,4.85,Disc,Drum,"Tubeless,Radial",5.0,355.1
1116,1116,jaipur,Petrol,Hatchback,"70,000",Manual,1,Maruti,Maruti Alto,2007,...,690kg,5 Speed,2WD,Manual,4.60,Disc,Drum,"Tubeless,Radial",5.0,355.1
1117,1117,jaipur,Petrol,SUV,"20,000",Manual,1,Renault,Renault Kiger,2022,...,1012,5 Speed,NaN,Electric,5.00,Disc,Drum,"Tubeless, Radial",5.0,405.0
1118,1118,jaipur,Petrol,Hatchback,"40,000",Manual,1,Maruti,Maruti Alto,2009,...,690kg,5 Speed,2WD,Manual,4.60,Disc,Drum,"Tubeless,Radial",5.0,355.1


In [4]:
import pandas as pd
import ast
import re
df=pd.read_excel("C:\project 3\kolkata_cars.xlsx")

# new_car_detail

df['new_car_detail'] = df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
new_car_details_df = pd.json_normalize(df['new_car_detail'])
new_car_details_df.insert(0, 'state', 'kolkata')
new_car_details_df.drop(columns=["it","owner","priceFixedText","priceActual","priceSaving","trendingText.imgUrl",
                          "trendingText.heading","trendingText.desc"],inplace=True)

def convert_numberic(value):
    new=value.replace("₹","").replace(',', '').strip()
    if "Lakh" in new:
        return float(new.replace('Lakh', '').strip())*1e5
    elif "Crore" in new:
        return float(new.replace("Crore","").strip())*1e7
    else:
        return float(new)
new_car_details_df["price"]=new_car_details_df["price"].apply(convert_numberic)

# car_overview


# Extracting 'top' key values into separate columns
def extract_keys(dict_list):
    result = {}
    for item in dict_list:
        result[item['key']] = item['value']
    return result

# Apply function and expand the dictionary into columns
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
car_overview = df["new_car_overview"].apply(lambda x: extract_keys(x['top'])).apply(pd.Series)

#convert dataformat


def extract_year(value):
    match = re.search(r'\b\d{4}\b', str(value))
    if match:
        # Extract the year and return it as an integer
        return int(match.group())
    return pd.NA  # Use pd.NA for missing values

# Apply the function to extract the year
car_overview['Registration Year'] = car_overview['Registration Year'].apply(extract_year)

# Convert the column datatype to 'Int64' (nullable integer type)
car_overview['Registration Year'] = car_overview['Registration Year'].astype('Int64')


def extract_number(text):
    # Ensure the input is a string
    if isinstance(text, str):
        match = re.search(r'\d+', text)
        if match:
            return int(match.group())
    return None


car_overview['Seats'] = car_overview['Seats'].apply(extract_number)


import pandas as pd
import re

def extract_kms(text):
    
    if isinstance(text, str):
        
        numeric_value = re.sub(r'[^\d,]', '', text)
        
        return int(numeric_value.replace(',', ''))
    return None


# Apply the function to the 'Distance' column
car_overview['Kms Driven'] = car_overview['Kms Driven'].apply(extract_kms)
car_overview.drop(columns=["Ownership"],inplace=True)
#null imputation integers and float
car_overview["Registration Year"]=car_overview["Registration Year"].fillna(2019)
car_overview["Seats"]=car_overview["Seats"].fillna(5.0)
car_overview["Kms Driven"]=car_overview["Kms Driven"].fillna(car_overview["Kms Driven"].mean())
car_overview["Year of Manufacture"]=car_overview["Year of Manufacture"].fillna(2019)
# df_car_fearures
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
result=[]
for sec in df["new_car_feature"]:
   
   
    df1 = pd.DataFrame([item['value'] for item in sec["top"]], columns=['Features'])

# Concatenate all the values into a single string and place it in a single row
    concatenated_features = ','.join(df1['Features'])
    result.append({'Features': concatenated_features})

df_car_fearures = pd.DataFrame(result)
# new_car_specs
df['new_car_specs'] = df['new_car_specs'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

def extract_specs(specs):
   
    # Extract from 'top'
    top_data = extract_keys(specs.get('top', []))
    
    # Extract from 'list' in 'data'
    list_data = {}
    if 'data' in specs:
        for section in specs['data']:
            list_data.update(extract_keys(section.get('list', [])))
    
    # Combine both dictionaries
    combined_data = {**top_data, **list_data}
    return combined_data

car_spec_df = df["new_car_specs"].apply(extract_specs).apply(pd.Series)
#convert dataformat
car_spec_df.drop(columns=["Ground Clearance Unladen","Seats","Values per Cylinder","Front Tread","Seating Capacity",
                          "Compression Ratio","BoreX Stroke","Gross Weight","Alloy Wheel Size","Rear Tread","Acceleration",
                          "Top Speed","Wheel Size","Value Configuration"
                          ],inplace=True)

def millimeter(value,mini):
    if pd.notna(value) and mini in value:
        value = value.replace(mini, "").strip()
        value = pd.to_numeric(value, errors='coerce')
        return value
    return pd.to_numeric(value, errors='coerce')

def meter(value,mini1,mini2,mini3):
    if pd.notna(value) and mini1 and mini2 and mini3 in value:
        value = value.replace(mini1, "").replace(mini2, "").replace(mini3, "").strip()
        value = pd.to_numeric(value, errors='coerce')
        return value
    return pd.to_numeric(value, errors='coerce')

car_spec_df["Mileage"] = car_spec_df["Mileage"].str.replace(" kmpl", "", regex=True)
car_spec_df["Mileage"] = pd.to_numeric(car_spec_df["Mileage"], errors='coerce')

car_spec_df["Torque"] = car_spec_df["Torque"].str.replace("Nm", "", regex=True).str.replace("nm", "", regex=True)
car_spec_df["Torque"] = pd.to_numeric(car_spec_df["Torque"], errors='coerce')

car_spec_df["Engine"] = car_spec_df["Engine"].str.replace("CC", "", regex=True)
car_spec_df["Engine"] = pd.to_numeric(car_spec_df["Engine"], errors='coerce')

car_spec_df["Displacement"]=pd.to_numeric(car_spec_df["Displacement"], errors='coerce')

car_spec_df["No of Cylinder"] = pd.to_numeric(car_spec_df["No of Cylinder"], errors='coerce')

car_spec_df["No Door Numbers"]=pd.to_numeric(car_spec_df["No Door Numbers"], errors='coerce')

car_spec_df["Cargo Volumn"]=car_spec_df["Cargo Volumn"].apply(lambda x: millimeter(x, "-litre"))

car_spec_df["Turning Radius"]=car_spec_df["Turning Radius"].apply(lambda x:meter(x, "metres","mtrs.","m"))


#null imputation
car_spec_df["Mileage"]=car_spec_df["Mileage"].fillna(19.7)
car_spec_df["Torque"]=car_spec_df["Torque"].fillna(1370)
car_spec_df["No of Cylinder"]=car_spec_df["No of Cylinder"].fillna(4.0)
car_spec_df["Length"]=car_spec_df["Length"].fillna(	4047.6)
car_spec_df["Width"]=car_spec_df["Width"].fillna(1709.4)
car_spec_df["Height"]=car_spec_df["Height"].fillna(1567.4)
car_spec_df["Wheel Base"]=car_spec_df["Wheel Base"].fillna(2523.9)
car_spec_df["Turning Radius"]=car_spec_df["Turning Radius"].fillna(5.0)
car_spec_df["No Door Numbers"]=car_spec_df["No Door Numbers"].fillna(5.0)
car_spec_df["Cargo Volumn"]=car_spec_df["Cargo Volumn"].fillna(355.1)
dfk = pd.concat([new_car_details_df, car_overview, df_car_fearures, car_spec_df], axis=1)

dfk.to_csv("kolkatanew.csv")
dfk=pd.read_csv("kolkatanew.csv")
dfk

<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
C:\Users\reach\AppData\Local\Temp\ipykernel_6936\2563280654.py:4: SyntaxWarning: invalid escape sequence '\p'
  df=pd.read_excel("C:\project 3\kolkata_cars.xlsx")


,Unnamed: 0,state,ft,bt,km,transmission,ownerNo,oem,model,modelYear,...,Kerb Weight,Gear Box,Drive Type,Steering Type,Turning Radius,Front Brake Type,Rear Brake Type,Tyre Type,No Door Numbers,Cargo Volumn
0,0,kolkata,Petrol,Sedan,"70,000",Automatic,3,Toyota,Toyota Camry,2014,...,1625kg,6 Speed,FWD,Power,5.50,Ventilated Disc,Solid Disc,"Tubeless,Radial",4.0,355.1
1,1,kolkata,Petrol,Hatchback,"23,981",Manual,1,Datsun,Datsun RediGO,2017,...,770,5-Speed,2WD,Power,4.70,Disc,Drum,NaN,5.0,222.0
2,2,kolkata,Petrol,SUV,"7,100",Automatic,1,Renault,Renault Kiger,2021,...,1055,5 Speed,NaN,Electric,5.00,Disc,Drum,"Tubeless, Radial",5.0,405.0
3,3,kolkata,Petrol,Hatchback,"71,574",Manual,2,Hyundai,Hyundai i20,2011,...,1020,5 Speed,FWD,Power,5.20,Disc,Drum,"Tubeless,Radial",5.0,355.1
4,4,kolkata,Diesel,SUV,"50,000",Automatic,2,Audi,Audi Q3,2014,...,1660kg,7-Speed S-Tronic,AWD,Power,5.90,Ventilated Disc,Drum,"Tubeless,Radial",5.0,355.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,1376,kolkata,Petrol,Hatchback,"10,000",Manual,1,Maruti,Maruti Celerio,2022,...,810,5-Speed,NaN,Electric,5.00,Ventilated Disc,Drum,"Tubeless, Radial",5.0,313.0
1377,1377,kolkata,Petrol,Hatchback,"1,20,000",Manual,1,Maruti,Maruti Alto 800,2014,...,720kg,5 Speed,FWD,Power,4.60,Solid Disc,Drum,Tubeless Tyres,5.0,355.1
1378,1378,kolkata,Petrol,Sedan,"50,000",Automatic,3,Mercedes-Benz,Mercedes-Benz C-Class,2011,...,1585kg,7 Speed,RWD,Power,5.42,Ventilated Disc,Solid Disc,"Tubeless,Radial",4.0,355.1
1379,1379,kolkata,Petrol,Hatchback,"40,000",Manual,1,Maruti,Maruti Ritz,2012,...,1030kg,5 Speed,FWD,Power,4.70,Ventilated Disc,Drum,"Tubeless,Radial",5.0,355.1


In [3]:
import pandas as pd
import ast
import re
df=pd.read_excel(r"C:\project 3\bangalore_cars.xlsx")

# new_car_detail

df['new_car_detail'] = df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
new_car_details_df = pd.json_normalize(df['new_car_detail'])
new_car_details_df.insert(0, 'state', 'bangalore')
new_car_details_df.drop(columns=["it","owner","priceFixedText","priceActual","priceSaving","trendingText.imgUrl",
                          "trendingText.heading","trendingText.desc"],inplace=True)

def convert_numberic(value):
    new=value.replace("₹","").replace(',', '').strip()
    if "Lakh" in new:
        return float(new.replace('Lakh', '').strip())*1e5
    elif "Crore" in new:
        return float(new.replace("Crore","").strip())*1e7
    else:
        return float(new)
new_car_details_df["price"]=new_car_details_df["price"].apply(convert_numberic)

# car_overview


# Extracting 'top' key values into separate columns
def extract_keys(dict_list):
    result = {}
    for item in dict_list:
        result[item['key']] = item['value']
    return result

# Apply function and expand the dictionary into columns
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
car_overview = df["new_car_overview"].apply(lambda x: extract_keys(x['top'])).apply(pd.Series)

#convert dataformat


def extract_year(value):
    match = re.search(r'\b\d{4}\b', str(value))
    if match:
        # Extract the year and return it as an integer
        return int(match.group())
    return pd.NA  # Use pd.NA for missing values

# Apply the function to extract the year
car_overview['Registration Year'] = car_overview['Registration Year'].apply(extract_year)

# Convert the column datatype to 'Int64' (nullable integer type)
car_overview['Registration Year'] = car_overview['Registration Year'].astype('Int64')


def extract_number(text):
    # Ensure the input is a string
    if isinstance(text, str):
        match = re.search(r'\d+', text)
        if match:
            return int(match.group())
    return None


car_overview['Seats'] = car_overview['Seats'].apply(extract_number)


import pandas as pd
import re

def extract_kms(text):
    
    if isinstance(text, str):
        
        numeric_value = re.sub(r'[^\d,]', '', text)
        
        return int(numeric_value.replace(',', ''))
    return None


# Apply the function to the 'Distance' column
car_overview['Kms Driven'] = car_overview['Kms Driven'].apply(extract_kms)
car_overview.drop(columns=["Ownership"],inplace=True)
#null imputation integers and float
car_overview["Registration Year"]=car_overview["Registration Year"].fillna(2019)
car_overview["Seats"]=car_overview["Seats"].fillna(5.0)
car_overview["Kms Driven"]=car_overview["Kms Driven"].fillna(car_overview["Kms Driven"].mean())
car_overview["Year of Manufacture"]=car_overview["Year of Manufacture"].fillna(2019)
# df_car_fearures
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
result=[]
for sec in df["new_car_feature"]:
   
   
    df1 = pd.DataFrame([item['value'] for item in sec["top"]], columns=['Features'])

# Concatenate all the values into a single string and place it in a single row
    concatenated_features = ','.join(df1['Features'])
    result.append({'Features': concatenated_features})

df_car_fearures = pd.DataFrame(result)
# new_car_specs
df['new_car_specs'] = df['new_car_specs'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

def extract_specs(specs):
   
    # Extract from 'top'
    top_data = extract_keys(specs.get('top', []))
    
    # Extract from 'list' in 'data'
    list_data = {}
    if 'data' in specs:
        for section in specs['data']:
            list_data.update(extract_keys(section.get('list', [])))
    
    # Combine both dictionaries
    combined_data = {**top_data, **list_data}
    return combined_data

car_spec_df = df["new_car_specs"].apply(extract_specs).apply(pd.Series)
#convert dataformat
car_spec_df.drop(columns=["Ground Clearance Unladen","Seats","Values per Cylinder","Front Tread","Seating Capacity",
                          "Compression Ratio","BoreX Stroke","Gross Weight","Alloy Wheel Size","Rear Tread","Acceleration",
                          "Top Speed","Wheel Size","Value Configuration"
                          ],inplace=True)

def millimeter(value,mini):
    if pd.notna(value) and mini in value:
        value = value.replace(mini, "").strip()
        value = pd.to_numeric(value, errors='coerce')
        return value
    return pd.to_numeric(value, errors='coerce')

def meter(value,mini1,mini2,mini3):
    if pd.notna(value) and mini1 and mini2 and mini3 in value:
        value = value.replace(mini1, "").replace(mini2, "").replace(mini3, "").strip()
        value = pd.to_numeric(value, errors='coerce')
        return value
    return pd.to_numeric(value, errors='coerce')

car_spec_df["Mileage"] = car_spec_df["Mileage"].str.replace(" kmpl", "", regex=True)
car_spec_df["Mileage"] = pd.to_numeric(car_spec_df["Mileage"], errors='coerce')

car_spec_df["Torque"] = car_spec_df["Torque"].str.replace("Nm", "", regex=True).str.replace("nm", "", regex=True)
car_spec_df["Torque"] = pd.to_numeric(car_spec_df["Torque"], errors='coerce')

car_spec_df["Engine"] = car_spec_df["Engine"].str.replace("CC", "", regex=True)
car_spec_df["Engine"] = pd.to_numeric(car_spec_df["Engine"], errors='coerce')

car_spec_df["Displacement"]=pd.to_numeric(car_spec_df["Displacement"], errors='coerce')

car_spec_df["No of Cylinder"] = pd.to_numeric(car_spec_df["No of Cylinder"], errors='coerce')

car_spec_df["No Door Numbers"]=pd.to_numeric(car_spec_df["No Door Numbers"], errors='coerce')

car_spec_df["Cargo Volumn"]=car_spec_df["Cargo Volumn"].apply(lambda x: millimeter(x, "-litre"))

car_spec_df["Turning Radius"]=car_spec_df["Turning Radius"].apply(lambda x:meter(x, "metres","mtrs.","m"))


#null imputation
car_spec_df["Mileage"]=car_spec_df["Mileage"].fillna(19.7)
car_spec_df["Torque"]=car_spec_df["Torque"].fillna(1370)
car_spec_df["No of Cylinder"]=car_spec_df["No of Cylinder"].fillna(4.0)
car_spec_df["Length"]=car_spec_df["Length"].fillna(	4047.6)
car_spec_df["Width"]=car_spec_df["Width"].fillna(1709.4)
car_spec_df["Height"]=car_spec_df["Height"].fillna(1567.4)
car_spec_df["Wheel Base"]=car_spec_df["Wheel Base"].fillna(2523.9)
car_spec_df["Turning Radius"]=car_spec_df["Turning Radius"].fillna(5.0)
car_spec_df["No Door Numbers"]=car_spec_df["No Door Numbers"].fillna(5.0)
car_spec_df["Cargo Volumn"]=car_spec_df["Cargo Volumn"].fillna(355.1)
dfb = pd.concat([new_car_details_df, car_overview, df_car_fearures, car_spec_df], axis=1)

dfb.to_csv("bangalorenew.csv")
dfb=pd.read_csv("bangalorenew.csv")
dfb

,Unnamed: 0,state,ft,bt,km,transmission,ownerNo,oem,model,modelYear,...,Kerb Weight,Gear Box,Drive Type,Steering Type,Turning Radius,Front Brake Type,Rear Brake Type,Tyre Type,No Door Numbers,Cargo Volumn
0,0,bangalore,Petrol,Hatchback,"1,20,000",Manual,3,Maruti,Maruti Celerio,2015,...,835kg,5 Speed,FWD,Power,4.7,Ventilated Disc,Drum,"Tubeless, Radial",5.0,355.1
1,1,bangalore,Petrol,SUV,"32,706",Manual,2,Ford,Ford Ecosport,2018,...,1242Kg,5 Speed,FWD,Power,5.3,Ventilated Disc,Drum,"Tubeless,Radial",4.0,355.1
2,2,bangalore,Petrol,Hatchback,"11,949",Manual,1,Tata,Tata Tiago,2018,...,1012kg,5 Speed,FWD,Power,5.0,Disc,Drum,Tubeless,5.0,355.1
3,3,bangalore,Petrol,Sedan,"17,794",Manual,1,Hyundai,Hyundai Xcent,2014,...,1180,5 Speed,FWD,Power,4.7,Disc,Drum,"Tubeless,Radial",4.0,355.1
4,4,bangalore,Diesel,SUV,"60,000",Manual,1,Maruti,Maruti SX4 S Cross,2015,...,1230Kg,5 Speed,FWD,Power,5.0,Ventilated Disc,Solid Disc,"Tubeless,Radial",5.0,355.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,1476,bangalore,Diesel,SUV,"1,56,039",Manual,2,Toyota,Toyota Fortuner,2012,...,1970kg,5 Speed,4WD,Power,5.9,Ventilated Disc,Drum,"Tubeless,Radial",5.0,355.1
1477,1477,bangalore,Petrol,Sedan,"56,000",Manual,2,Maruti,Maruti SX4,2008,...,1200kg,5 Speed,NaN,Power,5.3,Ventilated Disc,Drum,"Tubeless, Radial",4.0,355.1
1478,1478,bangalore,Petrol,Hatchback,"42,000",Manual,2,Honda,Honda Brio,2014,...,925kg,5 Speed,FWD,Power,4.5,Disc,Drum,"Tubeless,Radial",5.0,355.1
1479,1479,bangalore,Diesel,Hatchback,"93,003",Manual,1,Hyundai,Hyundai i20,2018,...,1515kg,6 Speed,FWD,Power,5.2,Disc,Drum,Tubeless,5.0,355.1


In [16]:
final= pd.concat([df, dfc, dfk, dfj,dfb], axis=0, ignore_index=True)

# Display the concatenated DataFrame (row-wise)
print("Row-wise Concatenation:")
print(final)

Row-wise Concatenation:
                                         new_car_detail  \
0     {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
1     {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
2     {'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...   
3     {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
4     {'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...   
...                                                 ...   
6879                                                NaN   
6880                                                NaN   
6881                                                NaN   
6882                                                NaN   
6883                                                NaN   

                                       new_car_overview  \
0     {'heading': 'Car overview', 'top': [{'key': 'R...   
1     {'heading': 'Car overview', 'top': [{'key': 'R...   
2     {'heading': 'Car overview', 'top': [{'key': 'R...   
3     {'heading': 'Car overview